In [1]:
from collections import namedtuple
from skimage import io
from PIL import Image
import time
import os
import re
from sklearn.cluster import *

import torch
from torchvision import models
from torchvision import transforms
from torch.optim import Adam
from torchvision import datasets
from torch.utils.data import DataLoader

<h1>Experiment : Measuring Efficacy of Gram Matrix in clustering different Styled images</h1>
<p>The result was very encouraging.Either of KMeans (at 1 & 4th layer) or Spectral clustering (at 2 & 3rd) algorithm of Sklearn earned near perfect clustering og Caligraphy vs Landscape drawings. The last layer's gram performed slightly poorly. But this maybe due to high dimensionality, which is 263k.</p>

In [2]:
class Vgg16(torch.nn.Module):
    def __init__(self,requires_grad=False):
        super(Vgg16,self).__init__()
        vgg_features = models.vgg16(pretrained=True).features
        self.slice1 = torch.nn.Sequential()
        self.slice2 = torch.nn.Sequential()
        self.slice3 = torch.nn.Sequential()
        self.slice4 = torch.nn.Sequential()
        for x in range(4):
            self.slice1.add_module(str(x),vgg_features[x])
        for x in range(4, 9):
            self.slice2.add_module(str(x),vgg_features[x])
        for x in range(9, 16):
            self.slice3.add_module(str(x), vgg_features[x])
        for x in range(16, 23):
            self.slice4.add_module(str(x), vgg_features[x])
            
        if not requires_grad:
            for param in self.parameters():
                param.requires_grad = False
                
    def forward(self, X):
        h = self.slice1(X)
        h_relu1_2 = h
        h = self.slice2(h)
        h_relu2_2 = h
        h = self.slice3(h)
        h_relu3_3 = h
        h = self.slice4(h)
        h_relu4_3 = h
        vgg_outputs = namedtuple("VggOutputs", ['relu1_2', 'relu2_2', 'relu3_3', 'relu4_3'])
        out = vgg_outputs(h_relu1_2, h_relu2_2, h_relu3_3, h_relu4_3)
        return out

In [3]:
def gram_matrix(y):
    (b, ch, h, w) = y.size()
    features = y.view(b, ch, w * h)
    features_t = features.transpose(1, 2)
    gram = features.bmm(features_t) / (ch * h * w)
    return gram

def mean_var(fs):
    (b, ch, h, w) = fs.size()
    features = fs.view(-1, w * h)
    m = features.mean(dim=1)
    v = features.var(dim=1)
    s = torch.stack((m,v)).view(1,-1)
    return s
    
def normalize_batch(batch):
    # normalize using imagenet mean and std
    mean = batch.new_tensor([0.485, 0.456, 0.406]).view(-1, 1, 1)
    std = batch.new_tensor([0.229, 0.224, 0.225]).view(-1, 1, 1)
    batch = batch.div_(255.0)
    return (batch - mean) / std

def gram(path,layer):
    img = Image.open(path)
    img = style_transform(img).unsqueeze(0).cuda()
    fs = vgg(normalize_batch(img))[layer]
    return mean_var(fs)

In [4]:
vgg = Vgg16(requires_grad=False).to('cuda')
style_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Lambda(lambda x: x.mul(255))
])
loss = torch.nn.MSELoss()

In [12]:
DIR = "d:/PastoralStyle/"
LAYER = 2
gs = {}
for file in os.listdir(DIR):
    gs[file] = torch.stack([gram(DIR+file,i) for i in range(4)]).view(1,-1)

RuntimeError: invalid argument 0: Sizes of tensors must match except in dimension 0. Got 1024 and 128 in dimension 2 at c:\programdata\miniconda3\conda-bld\pytorch_1533090623466\work\aten\src\thc\generic/THCTensorMath.cu:87

In [7]:
data = torch.cat([gs[file] for file in os.listdir(DIR)])
data.size()

torch.Size([35, 512])

In [8]:
cl = KMeans(n_clusters=2).fit(data) #SpectralClustering(n_clusters=2).fit(data) #

In [9]:
cl.labels_

array([0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0])

In [10]:
fs = os.listdir(DIR)
for i in range(len(cl.labels_)):
    if cl.labels_[i]==1:
        print(fs[i])

calligraphic-writing-in-sulus-and-nesih-scripts.jpg
calligraphy-exercise.jpg!Large.jpg
epitaph-1.jpg!Large.jpg
epitaph.jpg!Large.jpg
hilye-i-er-f.jpg!Large.jpg
levha-all-hu-vahdeh.jpg!Large.jpg
levha-dua.jpg!Large.jpg
levha-hadis-i-er-f.jpg!Large.jpg
levha-kelime-i-tevhid.jpg!Large.jpg
levha.jpg
s-l-s-karalama.jpg!Large.jpg
s-leymaniye-k-t-phanesi-1.jpg
serlevha-bakara-s-resi-nin-ilk-4-yeti.jpg!Large.jpg
serlevha-fatiha-s-resi-ve-bakara-s-resi-nin-ilk-4-yeti.jpg!Large.jpg
sultan-iv-mustafa.jpg
tekke-levha.jpg!Large.jpg
tile.jpg!Large.jpg


In [ ]:
len(gs)*1024

In [ ]:
im = "on-a-sussex-farm-1904.jpg!Large.jpg"
dis = []
for f in gs:
    ls = loss(gs[im],gs[f])
    dis.append((f,ls))
dis = sorted(dis,key=lambda x:x[1])